In [9]:
import spacy
import pandas as pd
import json

In [17]:



def xlsx_to_jsonl_with_entities(input_file, output_file):
    # Чтение Excel файла
    xls = pd.ExcelFile(input_file)
    sheets = xls.sheet_names
    
    # Список для хранения JSON Lines
    json_lines = []
    
    for sheet in sheets:
        # Чтение данных с текущего листа
        df = pd.read_excel(input_file, sheet_name=sheet)
        
        # Формирование JSON Lines
        for _, row in df.iterrows():
            text = row['Название товара']
            words = text.split()
            
            if words:
                first_word = words[0]
                start_idx = text.index(first_word)
                end_idx = start_idx + len(first_word)
                
                entities = [(start_idx, end_idx, sheet)]
                
                json_obj = {
                    "text": text,
                    "entities": entities
                }
                # print(json_obj, )
                json_lines.append(json_obj)
    # # Запись в JSON Lines файл
    # with open(output_file, 'w', encoding='utf-8') as f:
    #     for line in json_lines:
    #         f.write(json.dumps(line, ensure_ascii=False) + '\n')
    return json_lines
# Пример использования функции
input_file = 'dataset_LM_освещение.xlsx'  # Замените на ваше имя файла
output_file = 'output_with_entities.jsonl'
ner_markup_data = xlsx_to_jsonl_with_entities(input_file, output_file)
print(ner_markup_data[0])

{'text': 'ЛЮСТРА БРИЗ 6E27X6X60W 6X3W LED ХР ПУЛЬТ', 'entities': [(0, 6, 'люстры под лампу (интерьер внут')]}


In [18]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

In [19]:
from spacy.util import filter_spans

for training_example  in tqdm(ner_markup_data): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy") 
     

100%|██████████| 5279/5279 [00:00<00:00, 11828.39it/s]


In [21]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:

!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy 

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.47   10.29    5.87   41.64    0.10
  0     200         35.41   2026.89   87.42   87.17   87.67    0.87
  1     400         54.89    701.76   95.67   96.24   95.11    0.96
  2     600         41.20    489.26   96.29   96.29   96.29    0.96
  3     800         70.47    464.51   97.69   97.69   97.69    0.98
  4    1000        112.96    607.63   97.61   97.61   97.61    0.98
  6    1200        128.71    624.24   98.83   98.83   98.83    0.99
  7    1400         91.67    341.46   99.55   99.55   99.55    1.00
 10    1600        267.16    421.04 

In [24]:
nlp_ner = spacy.load("model-best")

In [28]:
# doc = nlp_ner(r"Люстра потолочная светодиодная «Quieto» 1905-2/2 с пультом управления 25 м² регулируемый белый свет цвет белый")
# doc = nlp_ner(r"Столб уличный Inspire Travis 45 см")
doc = nlp_ner(r"Светильник уличный настенный Inspire Lakko LED 1000 Лм 4000 К IP44 цвет серый")

colors = {"люстры под лампу (интерьер внут": "#F67DE3", "люстры светодиодные (интерьер в": "#7DF6D9", "уличный светильник под лампу (":"#a6e22d","уличный светильник светодиодный":"#9966cc","уличные фонари столбы (экстерье":"#318ce7"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)